# 절대 모멘텀

In [1]:
import pandas as pd
import numpy as np
import datetime

read_df = pd.read_csv('../data/us_etf_data/SPY.csv')
read_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,26.706757,1003200
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,26.896694,480500
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,26.953669,201300
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,27.238594,529400
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,27.352570,531500


In [2]:
price_df = read_df.loc[:,['Date','Adj Close']].copy()
price_df.head()

,Date,Adj Close
0,1993-01-29,26.706757
1,1993-02-01,26.896694
2,1993-02-02,26.953669
3,1993-02-03,27.238594
4,1993-02-04,27.352570


book은 거래내역 및 수익률을 계산하려고 하는거니깐.
trading에 쓸 price df 는 ab_mom_df를 만들어야겠다.

In [3]:
price_df['STD_YM']= price_df['Date'].map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d')\
                                         .strftime('%Y-%m'))
price_df.head()

,Date,Adj Close,STD_YM
0,1993-01-29,26.706757,1993-01
1,1993-02-01,26.896694,1993-02
2,1993-02-02,26.953669,1993-02
3,1993-02-03,27.238594,1993-02
4,1993-02-04,27.352570,1993-02


In [4]:
month_list = price_df['STD_YM'].unique()

In [5]:
month_last_df = pd.DataFrame()
for m in month_list:
    # 기준년월에 맞는 인덱스의 가장 마지막 날짜 row를 데이터 프레임에 추가한다
    month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1]\
                                                       , : ]
                                        )

month_last_df.set_index(['Date'],inplace=True)
month_last_df.head()

AttributeError: 'DataFrame' object has no attribute 'append'

In [6]:
month_last_df['BF_1M_Adj Close'] = month_last_df.shift(1)['Adj Close']
month_last_df['BF_12M_Adj Close'] = month_last_df.shift(12)['Adj Close']
month_last_df.fillna(0, inplace=True)
month_last_df.head(15)

,Adj Close,STD_YM,BF_1M_Adj Close,BF_12M_Adj Close
Date,,,,
1993-01-29,26.706757,1993-01,0.000000,0.000000
1993-02-26,26.991682,1993-02,26.706757,0.000000
1993-03-31,27.596252,1993-03,26.991682,0.000000
1993-04-30,26.890142,1993-04,27.596252,0.000000
1993-05-28,27.615356,1993-05,26.890142,0.000000
1993-06-30,27.714964,1993-06,27.615356,0.000000
1993-07-30,27.580431,1993-07,27.714964,0.000000
1993-08-31,28.637531,1993-08,27.580431,0.000000
1993-09-30,28.429131,1993-09,28.637531,0.000000


In [7]:
book = price_df.copy()
book.set_index(['Date'],inplace=True)
book['trade'] = ''
book.head()

,Adj Close,STD_YM,trade
Date,,,
1993-01-29,26.706757,1993-01,
1993-02-01,26.896694,1993-02,
1993-02-02,26.953669,1993-02,
1993-02-03,27.238594,1993-02,
1993-02-04,27.352570,1993-02,


In [7]:
#trading 부분.
ticker = 'SPY'
for x in month_last_df.index:
    signal = ''
    # 절대 모멘텀을 계산한다. 
    momentum_index = month_last_df.loc[x,'BF_1M_Adj Close'] / month_last_df.loc[x,'BF_12M_Adj Close'] -1
    # 절대 모멘텀 지표 True / False를 판단한다.
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf))\
    else False \
    and True
    if flag :
        signal = 'buy ' + ticker # 절대 모멘텀 지표가 Positive이면 매수 후 보유.
    print('날짜 : ',x,' 모멘텀 인덱스 : ',momentum_index, 'flag : ',flag ,'signal : ',signal)
    book.loc[x:,'trade'] = signal

In [8]:
book.tail(25)

NameError: name 'book' is not defined

In [10]:
def returns(book, ticker):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy '+ ticker and book.shift(1).loc[i,'trade'] == '' :     # long 진입
            buy = book.loc[i, 'Adj Close']
            print('진입일 : ',i, 'long 진입가격 : ', buy)
        elif book.loc[i, 'trade'] == 'buy '+ ticker and book.shift(1).loc[i,'trade'] == 'buy '+ ticker :
            # 보유중  
            current = book.loc[i, 'Adj Close']
            rtn = (current - buy) / buy + 1
            book.loc[i, 'return'] = rtn
            
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy '+ticker:     # long 청산
            sell = book.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ',i, 'long 진입가격 : ', buy, ' |  long 청산가격 : ', \
                  sell, ' | return:', round(rtn, 4))

        if book.loc[i, 'trade'] == '':     # zero position
            buy = 0.0
            sell = 0.0
            current = 0.0

    acc_rtn = 1.0
    for i in book.index:
        if book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy '+ticker:     # long 청산시
            rtn = book.loc[i, 'return']
            acc_rtn = acc_rtn * rtn # 누적수익률 계산
            book.loc[i:, 'acc return'] = acc_rtn

    print ('Accunulated return :', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [11]:
returns(book,ticker)

진입일 :  1994-01-31 long 진입가격 :  30.045222999999996
청산일 :  1994-12-30 long 진입가격 :  30.045222999999996  |  long 청산가격 :  29.148033  | return: 0.9701
진입일 :  1995-02-28 long 진입가격 :  31.357141
청산일 :  2000-12-29 long 진입가격 :  31.357141  |  long 청산가격 :  92.002388  | return: 2.934
진입일 :  2001-02-28 long 진입가격 :  86.926689
청산일 :  2001-03-30 long 진입가격 :  86.926689  |  long 청산가격 :  82.055542  | return: 0.944
진입일 :  2003-07-31 long 진입가격 :  72.26248199999999
청산일 :  2008-02-29 long 진입가격 :  72.26248199999999  |  long 청산가격 :  105.568001  | return: 1.4609
진입일 :  2009-10-30 long 진입가격 :  85.141289
청산일 :  2011-10-31 long 진입가격 :  85.141289  |  long 청산가격 :  107.309486  | return: 1.2604
진입일 :  2011-11-30 long 진입가격 :  106.873421
청산일 :  2012-01-31 long 진입가격 :  106.873421  |  long 청산가격 :  112.998047  | return: 1.0573
진입일 :  2012-02-29 long 진입가격 :  117.902817
청산일 :  2015-10-30 long 진입가격 :  117.902817  |  long 청산가격 :  193.002396  | return: 1.637
진입일 :  2015-11-30 long 진입가격 :  193.707825
청산일 :  2016-02-29 long 진입가격 : 

10.9953

In [12]:
book.loc['1994-11-31':,]

,Adj Close,STD_YM,trade,return,acc return
Date,,,,,
1994-12-01,28.649719,1994-12,buy SPY,0.953553,NaN
1994-12-02,28.917469,1994-12,buy SPY,0.962465,NaN
1994-12-05,28.947224,1994-12,buy SPY,0.963455,NaN
1994-12-06,28.967051,1994-12,buy SPY,0.964115,NaN
1994-12-07,28.758799,1994-12,buy SPY,0.957184,NaN
...,...,...,...,...,...
2019-06-18,290.984741,2019-06,buy SPY,1.053854,10.995336
2019-06-19,291.641541,2019-06,buy SPY,1.056232,10.995336
2019-06-20,294.427979,2019-06,buy SPY,1.066324,10.995336


### 전체코드

In [13]:
import pandas as pd
import numpy as np
import datetime

read_df = pd.read_csv('./data/us_etf_data/SPY.csv')
price_df = read_df.loc[:,['Date','Adj Close']].copy()
price_df['STD_YM']= price_df['Date'].map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d')\
                                         .strftime('%Y-%m'))
month_list = price_df['STD_YM'].unique()

month_last_df = pd.DataFrame()
for m in month_list:
    month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1]\
                                                       , : ]
                                        )
month_last_df.set_index(['Date'],inplace=True)
month_last_df['BF_1M_Adj Close'] = month_last_df.shift(1)['Adj Close']
month_last_df['BF_12M_Adj Close'] = month_last_df.shift(12)['Adj Close']
month_last_df.fillna(0, inplace=True)

book = price_df.copy()
book.set_index(['Date'],inplace=True)
book['trade'] = ''

#trading 부분.
ticker = 'SPY'
for x in month_last_df.index:
    signal = ''
    momentum_index = month_last_df.loc[x,'BF_1M_Adj Close'] / month_last_df.loc[x,'BF_12M_Adj Close'] -1
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf))\
    else False \
    and True
    if flag :
        signal = 'buy ' + ticker
    print('날짜 : ',x,' 모멘텀 인덱스 : ',momentum_index, 'flag : ',flag ,'signal : ',signal)
    book.loc[x:,'trade'] = signal
    
returns(book,ticker)

FileNotFoundError: [Errno 2] File ./data/us_etf_data/SPY.csv does not exist: './data/us_etf_data/SPY.csv'